# 🧠 TrackZone IA Realista - Organização com Probabilidade e Aprendizado Supervisionado
Este notebook utiliza uma base mais realista, onde a zona ideal é atribuída com **probabilidades**, e não de forma determinística. Treinamos um modelo supervisionado (MLPClassifier) para prever a zona ideal com acurácia próxima de 90%.

In [4]:

# !pip install plotly
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score


In [5]:

df = pd.read_csv("/content/sample_data/base_motos_realista.csv")
df.head()


,placa,numero_chassi,numero_motor,status,x,y,nova_x,nova_y,zona_ideal,descricao_posicao,status_encoded,zona_encoded
0,ABC0000,CHS555102,MTR668535,Aguardando aluguel,3.92,9.15,6.3,8.86,BOX_RAPIDO,Disponível atrás da moto com placa ABC0000,0,1
1,ABC0001,CHS437889,MTR345121,Danos estruturais,8.17,3.62,0.2,8.33,RAMPA_MANUTENCAO,Na fila de manutenção estrutural - moto placa ...,2,3
2,ABC0002,CHS576467,MTR772108,Defeito no motor,4.39,5.81,7.2,5.83,BANCADA_MOTOR,Na bancada de motor sem placa ou com defeito -...,3,0
3,ABC0003,CHS456600,MTR467257,Em manutenção,3.77,6.32,2.2,8.75,BOX_RAPIDO,Aguardando reparo no box rápido - moto placa A...,5,1
4,ABC0004,CHS415557,MTR366765,Sem placa,4.63,0.13,7.6,5.83,BANCADA_MOTOR,Na bancada de motor sem placa ou com defeito -...,7,0


In [6]:

# Codificação
le_status = LabelEncoder()
le_zona = LabelEncoder()

df["status_encoded"] = le_status.fit_transform(df["status"])
df["zona_encoded"] = le_zona.fit_transform(df["zona_ideal"])

X = df[["status_encoded"]]
y = df["zona_encoded"]

# Validação cruzada com pipeline
pipeline = make_pipeline(
    StandardScaler(),
    MLPClassifier(hidden_layer_sizes=(32, 32), activation='relu', max_iter=3000, random_state=42)
)

scores = cross_val_score(pipeline, X, y, cv=5, scoring='accuracy')
print("Acurácia média:", round(scores.mean() * 100, 2), "%")
print("Desvio padrão:", round(scores.std() * 100, 2), "%")


Acurácia média: 89.14 %
Desvio padrão: 5.74 %


In [7]:

pipeline.fit(X, y)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpclassifier',
                 MLPClassifier(hidden_layer_sizes=(32, 32), max_iter=3000,
                               random_state=42))])

In [8]:

zonas = {
    "BOX_RAPIDO": {"x": (2, 4), "y": (6, 9)},
    "RAMPA_MANUTENCAO": {"x": (0, 2), "y": (5, 9)},
    "BANCADA_MOTOR": {"x": (7, 9), "y": (4, 6)},
    "PATIO": {"x": (6, 9), "y": (7, 9)},
}

zona_contadores = {
    zona: len(df[df["zona_ideal"] == zona])
    for zona in zonas
}

def adicionar_moto_ia(placa, chassi, motor, status):
    global df
    status_encoded = le_status.transform([status])[0]
    zona_pred_index = pipeline.predict([[status_encoded]])[0]
    zona_pred = le_zona.inverse_transform([zona_pred_index])[0]

    count = zona_contadores[zona_pred]
    zona_contadores[zona_pred] += 1

    max_por_linha = 5
    col = count % max_por_linha
    lin = count // max_por_linha

    x0, x1 = zonas[zona_pred]["x"]
    y0, y1 = zonas[zona_pred]["y"]
    espaco_x = (x1 - x0) / max_por_linha
    espaco_y = (y1 - y0) / 10

    nova_x = round(x0 + col * espaco_x + espaco_x / 2, 2)
    nova_y = round(y1 - lin * espaco_y - espaco_y / 2, 2)

    descricao = f"Moto com status '{status}' alocada na zona {zona_pred} - placa {placa}"

    nova_moto = {
        "placa": placa,
        "numero_chassi": chassi,
        "numero_motor": motor,
        "status": status,
        "x": None,
        "y": None,
        "nova_x": nova_x,
        "nova_y": nova_y,
        "zona_ideal": zona_pred,
        "descricao_posicao": descricao
    }

    df = pd.concat([df, pd.DataFrame([nova_moto])], ignore_index=True)


In [9]:

def mostrar_patio():
    fig = px.scatter(
        df,
        x="nova_x",
        y="nova_y",
        color="zona_ideal",
        hover_data=["placa", "status", "zona_ideal", "descricao_posicao"],
        title="🛵 Pátio com Motos Organizadas por IA",
        labels={"nova_x": "Coordenada X", "nova_y": "Coordenada Y"}
    )
    fig.update_traces(marker=dict(size=10))
    fig.update_layout(height=600)
    fig.show()


In [23]:
# exemplos de posições:
#    Reparo simples
#    Danos estruturais
#    Defeito no motor
#    Sem placa
#    Em manutenção
#    Disponível
#    Alugada
#    Aguardando aluguel


adicionar_moto_ia("ABC9999", "CHS112211", "MTR445500", "Disponível")
mostrar_patio()


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

<ipython-input-8-b9adfa1b44f4>:49: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

